### URL To Image

In [3]:
from urllib.request import urlopen
def url_to_image(url, readFlag=cv2.IMREAD_COLOR):
    resp = urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, readFlag)
    return image

### Data Extraction from Image

In [8]:
from PIL import Image
from pytesseract import pytesseract
import numpy as np
import cv2

#Define path to tessaract.exe
path_to_tesseract = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
pytesseract.tesseract_cmd = path_to_tesseract

#Define path to image
path_to_image = 'IR-M-S-8890.jpg'
path_to_image1 = 'IR-E-U-0456.jpg'
path_to_image2 = 'IR-E-U-0037.jpg'
def image_to_data(path_to_image):
#     image = cv2.imread(path_to_image)
    image=url_to_image(path_to_image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    invert = 255 - thresh
    text = pytesseract.image_to_string(invert, lang='eng', config='--psm 6')
    print(text)
    arr=[]
    for line in text.splitlines():
        arr.append(line)
    arr=np.array(arr[-5:])
    x = arr[3].split(" ")
    
    if(len(x) != 19):
        x = arr[2].split(" ")
#     print(x)
    a=np.array(x)
    a=a[2:]
    temp=[]
    for i in a:
        f=i.replace(",","")
        temp.append(f)
    a=np.array(temp)
    a = a.astype(float)
#     temp=[]
    final=a
#     for i in final:
#         j=float(i)
#         temp.append(j)
#     final=np.array(temp)
    print(final)
    return final

# a=image_to_data(path_to_image)
b=image_to_data('https://www.nirfindia.org/nirfpdfcdn/2021/graph/Engineering/IR-E-U-0456.jpg')
# c=image_to_data(path_to_image2)

Indian Institute of Technology Madras (IR-E-U-0456)
2 oS
i
ss FSR FOE FRU PU QP IPR FPPP GPH GUE MS GPHD RD WD Escs PCS PR
TLR RP Go or PR
| ss FSR FQE FRU PU QP IPR FPPP GPH GUE MS GPHD RD WD ESCS PCS PR
| Score 18.50 30.00 17.88 29.09 34.43 37.00 15.00 10.00 28.50 15.00 21.10 17.32 21.43 13.57 7.43 20.00 100.00
| Total 20.00 30.00 20.00 30.00 35.00 40.00 15.00 10.00 40.00 15.00 25.00 20.00 30.00 30.00 20.00 20.00 100.00

[ 18.5   30.    17.88  29.09  34.43  37.    15.    10.    28.5   15.
  21.1   17.32  21.43  13.57   7.43  20.   100.  ]


### Web Scrapping

In [10]:
from autoscraper import AutoScraper
url="https://www.nirfindia.org/2021/EngineeringRanking.html"
wanted_list=["IR-E-U-0456","Indian Institute of Technology Madras","https://www.nirfindia.org/nirfpdfcdn/2021/graph/Engineering/IR-E-U-0456.jpg","95.47","96.43","81.92","62.44","100.00","https://www.nirfindia.org/nirfpdfcdn/2021/pdf/Engineering/IR-E-U-0456.pdf","90.19"]
scraper=AutoScraper()
result=scraper.build(url,wanted_list)
rule=scraper.get_result_similar(url,grouped=True)
keys=list(rule.keys())
scraper.set_rule_aliases({keys[0]:'Institute ID',keys[1]:'Name',keys[2]:'image',keys[3]:'TLR',keys[4]:'RPC',keys[5]:'GO',keys[6]:'OI',keys[7]:'PERCEPTION',keys[8]:'pdf',keys[9]:'Score'})
scraper.keep_rules([keys[0],keys[1],keys[2],keys[3],keys[4],keys[5],keys[6],keys[7],keys[8],keys[9]])
scraper.save('nirf')

In [11]:
result=scraper.get_result_similar(url,group_by_alias=True)
(result['Name'])

['Indian Institute of Technology Madras',
 'Indian Institute of Technology Delhi',
 'Indian Institute of Technology Bombay',
 'Indian Institute of Technology Kanpur',
 'Indian Institute of Technology Kharagpur',
 'Indian Institute of Technology Roorkee',
 'Indian Institute of Technology Guwahati',
 'Indian Institute of Technology Hyderabad',
 'National Institute of Technology Tiruchirappalli',
 'National Institute of Technology Karnataka',
 'Indian Institute of Technology (Indian School of Mines)',
 'Vellore Institute of Technology',
 'Indian Institute of Technology Indore',
 'Indian Institute of Technology (Banaras Hindu University) Varanasi',
 'Institute of Chemical Technology',
 'Amrita Vishwa Vidyapeetham',
 'Jadavpur University',
 'Anna University',
 'Indian Institute of Technology Ropar',
 'National Institute of Technology Rourkela',
 'Indian Institute of Technology Patna',
 'Indian Institute of Technology Gandhinagar',
 'National Institute of Technology Warangal',
 'Thapar Insti

In [50]:
university_2021=scraper.get_result_similar('https://www.nirfindia.org/2021/UniversityRanking.html',group_by_alias=True)
len(university_2021['Score'])

100

In [51]:
university_2021=scraper.get_result_similar('https://www.nirfindia.org/2021/ManagementRanking.html',group_by_alias=True)
len(university_2021['OI'])

75

In [12]:
management_2020=scraper.get_result_similar('https://www.nirfindia.org/2020/ManagementRanking.html',group_by_alias=True)
(management_2020['Score'])

['82.75',
 '81.32',
 '80.39',
 '73.85',
 '70.43',
 '69.96',
 '69.04',
 '67.19',
 '67.11',
 '65.95',
 '65.76',
 '62.87',
 '62.87',
 '61.77',
 '60.79',
 '60.53',
 '59.57',
 '56.93',
 '56.12',
 '55.97',
 '55.91',
 '55.82',
 '55.80',
 '55.44',
 '55.21',
 '55.01',
 '54.85',
 '54.65',
 '53.71',
 '53.56',
 '52.55',
 '52.15',
 '52.07',
 '51.41',
 '50.49',
 '49.51',
 '48.27',
 '48.27',
 '47.47',
 '47.38',
 '47.35',
 '47.22',
 '47.11',
 '46.75',
 '46.71',
 '46.69',
 '46.65',
 '46.63',
 '46.58',
 '46.51',
 '46.40',
 '46.22',
 '46.16',
 '46.14',
 '45.89',
 '45.50',
 '45.22',
 '45.13',
 '44.81',
 '44.71',
 '44.38',
 '44.26',
 '44.02',
 '44.02',
 '43.97',
 '43.73',
 '43.45',
 '43.40',
 '43.38',
 '43.30',
 '43.24',
 '42.70',
 '42.56',
 '42.47',
 '42.31']

In [56]:
architecture_2019=scraper.get_result_similar('https://www.nirfindia.org/2019/ArchitectureRanking.html',group_by_alias=True)
len(architecture_2019['TLR'])

15

In [65]:
engineering_2017=scraper.get_result_similar('https://www.nirfindia.org/2017/EngineeringRanking.html',group_by_alias=True)
len(engineering_2017['pdf'])

100